In [1]:
import flatbuffers

In [8]:
import sys
from tflite import Model

def main(model_path):
    with open(model_path, 'rb') as f:
        buffer_data = f.read()

    model = Model.GetRootAsModel(buffer_data, 0)
    opcodes = [model.OperatorCodes(i) for i in range(model.OperatorCodesLength())]

    unsupported_ops = set()

    for subgraph in range(model.SubgraphsLength()):
        subgraph_obj = model.Subgraphs(subgraph)

        for op_idx in range(subgraph_obj.OperatorsLength()):
            op_obj = subgraph_obj.Operators(op_idx)
            opcode_index = op_obj.OpcodeIndex()
            opcode = opcodes[opcode_index]

            if opcode.BuiltinCode() == 0:  # BuiltinCode 0 means it's a custom operation
                custom_op_name = opcode.CustomCode()
                if custom_op_name:
                    unsupported_ops.add(custom_op_name.decode('utf-8'))
                else:
                    unsupported_ops.add(f"Unknown custom op at index {opcode_index}")

    print("Unsupported operations:")
    for op in unsupported_ops:
        print(f"- {op}")

main(model_path='tflite_tests/heylol_midi.tflite')

Unsupported operations:
- Unknown custom op at index 7
